### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd 'drive/Othercomputers/My Laptop (1)/MY COLLEGE/ISI KOLKATA/PROJECTS/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

### Importing necessary libraries

In [3]:
!pip install phate
!pip install scprep

In [4]:
import numpy as np
import phate as ph
import pandas as pd
import scprep as scp
import matplotlib.pyplot as plt

import os

In [5]:
import TPHATE # Temporal PHATE
# 2 Modifications over Vanilla PHATE(Check https://github.com/KrishnaswamyLab/PHATE):
# (a) Calculation of Temporal Affinity Matrix
# (b) Changing the inputs of the Vanilla PHATE class to take the above matrix as input
# (c) Calculation of the Resultant Diffusion Operator as the product of the Vanilla PHATE Diffusion Operator and Temporal Affinity Matrix

In [6]:
!ls ./YOUNG

data_537_lOFC.npy  data_INS.npy   data_Nac.npy	      data_TP.npy
data_ACC.npy	   data_IPL.npy   data_PCC.npy	      data_vlPFC.npy
data_AMY.npy	   data_lOFC.npy  data_PCG.npy	      VC.xlsx
data_Cau.npy	   data_mCC.npy   data_Prec.npy       vlPFC.xlsx
data_dlPFC.npy	   data_mOFC.npy  data_pSTS.xlsx.npy  Voxel_BOLD.xlsx
data_dmPFC.npy	   data_mPFC.npy  data_Put.xlsx.npy   Whole_BOLD_20.xlsx


### Importing Data

We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS



In [14]:
file_names = [name for name in os.listdir('./YOUNG') if name.endswith("npy") and ("537" not in name)]

In [15]:
data = list()
for file_name in file_names:
  data.append(np.load('YOUNG/'+file_name))

We now find the BOLD data for all voxel and all the TRs averaged over the subjects.

In [17]:
data_avg = [np.mean(dat, axis=2) for dat in data]

In [18]:
data_avg[0].shape

(189, 1700)

### T-PHATE Representation

Here we apply Temporal PHATE Algorithm on the average young subjects BOLD data for each ROI. It differs from the Vanilla PHATE in the following way:
- Temporal Affinity Probability Matrix is calculated by:
  - Estimating the Autocorrelation Function for each voxel's Time Series with lags $0, 1, \dots, TR_{final}-1$.
  - Averaging the Autocorrelation Function($acf$) over the voxels.
  - Finding the first position(lag), where the $acf$ is $0$. Take this as $lag_{max}$.
  - Form the Affinity Matrix as $A$, where $A(i,j) = acf(i,j)$, whenever $0<|i-j|\leq lag_{max}$ and $0$ o.w.
  - Row-Normalize $A$ to obtain the Temporal Affinity Probability Matrix $P_T$
- Modifying the Diffusion Operator($P_D$) of the Vanilla PHATE to take into account the $P_T$ matrix by:
  - Modified Diffusion Operator = $P_D\cdot P_T$
- Carrying on the same operations as in Vanilla PHATE.

Disclaimer:
- The $acf$ function is not smoothed and the $lag_{max}$ is the first value of lag where the $acf$ drops below 0.
- $acf(i,j) = acf(lag=|i-j|)$ i.e. by assumption of stationary time series.

In [59]:
def phate_rep(data):
  # We find the temporal affinity matrix --- The Last Function in the TPHATE.py File
  P_T = TPHATE.temporal_affinity_matrix(data)

  # We now instantiate the PHATE operator(matrix)/object of the PHATE class
  phate_op = TPHATE.PHATE(P_T, random_state=43, verbose=0)
  # PHATE requires Voxel x TR Data
  phate_embd =  phate_op.fit_transform(data)

  return phate_embd

In [60]:
phate_embd = [phate_rep(data) for data in data_avg]

In [62]:
def plot_phate_rep(embd, c, ax=None, title="", legend_title="", save_file_name=None):
  scp.plot.scatter2d(embd, ax=ax,  c = c, label_prefix='T-PHATE ', title=title, legend_title=legend_title, figsize=(10, 8), filename=save_file_name)

In [63]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_rep(embd, list(range(data.shape[0])), None, "T-PHATE Represenation of "+ name, "TRs", "Plots/PHATE2-v-PHATE1-"+name+".png")

I have color coded each set of Ascent and Descent below, i.e.
- 0 indicates those TRs which are neither ascent and descent. 
- 1 indicates those TRs which are in A1 and D1.
- 2 indicates those TRs which are in A2 and D2.
- 3 indicates those TRs which are in A3 and D3.
- 4 indicates those TRs which are in A4 and A4
- 5 indicates those TRs wiich are in A5 and D5.

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_rep(embd, [0]*56+[1]*17+[2]*17+[0]*44+[3]*21+[4]*19+[5]*15, None, "T-PHATE Represenation of "+ name, "A&D TRs", "Plots/PHATE2-v-PHATE1-5-AnD-"+name+".png")

I have color coded each set of Ascent and Descent below, i.e.
- 0 indicates those TRs which are neither ascent and descent. 
- 1 indicates those TRs which are in Ascents.
- 2 indicates those TRs which are in Descents.

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_rep(embd, [0]*56+[1]*12+[2]*5+[1]*9+[2]*8+[0]*44+[1]*9+[2]*12+[1]*13+[2]*6+[1]*7+[2]*8, None, "T-PHATE Represenation of "+ name, "A&D TRs", "Plots/PHATE2-v-PHATE1-2-AnD-"+name+".png")

Below, we plot the PHATE 1 and PHATE 2 components wrt TRs.

In [68]:
def plot_phate_time_rep(embd, phate_comp=1, c=None, ax=None, title="", legend_title="", save_file_name=None):
  plot_data = np.concatenate((np.arange(embd.shape[0]).reshape(-1,1), embd[:,(phate_comp-1)].reshape(-1,1)), axis=1)
  scp.plot.scatter2d(plot_data, ax=ax,  c = c, xlabel='TRs', ylabel='T-PHATE '+str(phate_comp), title=title, legend_title=legend_title, figsize=(10, 8), filename=save_file_name)

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_rep(embd, 1, [0]*56+[1]*12+[2]*5+[1]*9+[2]*8+[0]*44+[1]*9+[2]*12+[1]*13+[2]*6+[1]*7+[2]*8, None, "T-PHATE over time of "+ name, "A&D TRs", "Plots/PHATE1-v-TR-2-AnD-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_rep(embd, 1, [0]*56+[1]*17+[2]*17+[0]*44+[3]*21+[4]*19+[5]*15, None, "T-PHATE over time of "+ name, "A&D TRs", "Plots/PHATE1-v-TR-5-AnD-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_rep(embd, 2, [0]*56+[1]*17+[2]*17+[0]*44+[3]*21+[4]*19+[5]*15, None, "T-PHATE over time of "+ name, "A&D TRs", "Plots/PHATE2-v-TR-5-AnD-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_rep(embd, 2, [0]*56+[1]*12+[2]*5+[1]*9+[2]*8+[0]*44+[1]*9+[2]*12+[1]*13+[2]*6+[1]*7+[2]*8, None, "T-PHATE over time of "+ name, "A&D TRs", "Plots/PHATE2-v-TR-2-AnD-"+name+".png")

Below, we show the PHATE 1 and PHATE 2 components wrt TRs with concatenated Ascent and Descent TRs.

In [77]:
def plot_phate_time_line_rep(embd, t, phate_comp=1, title="", filename=""):
  plt.figure(figsize=(10,8))
  plt.plot(np.arange(len(t))+1, embd[t,(phate_comp-1)])
  plt.title(title)
  plt.ylabel('T-PHATE '+str(phate_comp))
  plt.xlabel('TRs')
  plt.savefig(filename)

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_line_rep(embd, list(range(56, 68))+list(range(73, 82))+list(range(134, 143))+list(range(155, 168))+list(range(174, 181)), 1, "T-PHATE over concatenated ascent time of "+ name, "Plots/PHATE1-v-A-TR-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_line_rep(embd, list(range(56, 68))+list(range(73, 82))+list(range(134, 143))+list(range(155, 168))+list(range(174, 181)), 2, "T-PHATE over concatenated ascent time of "+ name, "Plots/PHATE1-v-A-TR-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_line_rep(embd, list(range(67, 73))+list(range(85, 90))+list(range(145, 155))+list(range(168, 175))+list(range(180, 189)), 1, "T-PHATE over concatenated descent time of "+ name, "Plots/PHATE1-v-D-TR-"+name+".png")

In [ ]:
for name, embd in zip(ROI_names, phate_embd):
  plot_phate_time_line_rep(embd, list(range(67, 73))+list(range(85, 90))+list(range(145, 155))+list(range(168, 175))+list(range(180, 189)), 2, "T-PHATE over concatenated descent time of "+ name, "Plots/PHATE1-v-D-TR-"+name+".png")

In [ ]:
!ls ./Plots/